## PSF Leakage Plots

This notebook creates all the plots of the PSF leakage: the scale-dependent $\alpha$ parameter, $\rho$ and $\tau$ statistics (from Guerrini et al. 2024) as well as the covariance matrix of the $\tau_0$ ad $\tau_2$ data vectors.

In [1]:
import matplotlib
from astropy.io import fits
import matplotlib.pyplot as plt
import uncertainties
import yaml
import os
import numpy as np

plt.rcParams.update({'font.size': 25,'figure.figsize':[12,7]})
plt.rcParams.update({"text.usetex": True})
plt.rc('mathtext', fontset='stix')
plt.rc('font', family='serif')

# Define path to data files  (NOTE: YOU MUST HAVE RAN THE cosmo_val.py PIPELINE!)
cosmoval_data_dir = "/n23data1/n06data/lgoh/scratch/UNIONS/sp_validation/notebooks/cosmo_val/output"
ver = "SP_v1.4.5"


### Plot the $\alpha(\theta)$ leakage

In [ ]:
# Read in the text file of the 2PCFs

alpha_fname = f"{cosmoval_data_dir}/leakage_{ver}/alpha_leakage.txt"
print(f"Reading xi_sys's from {alpha_fname}")

alphas = np.loadtxt(alpha_fname)
theta = alphas[:,0]
alpha = alphas[:,1]
var_alpha = alphas[:,2]


In [ ]:

# Plot the xi_+sys
ax1 = plt.subplot(111)
ax1.tick_params(axis='both', which='both', direction='in', length=6, width=1,
                top=True, bottom=True, left=True, right=True)
ax1.yaxis.minorticks_on()

ax1.errorbar(theta, alpha, yerr=var_alpha, 
                fmt='o', markersize=4, capsize=2, capthick=1.5, ls = 'solid', lw=1.8,
                color='darkviolet')
ax1.axhline(0,ls='dashed',lw=1, color='grey')
ax1.text(0.85, 0.15, r'$1-1$', transform=ax1.transAxes,
         bbox=dict(facecolor='white', edgecolor='black', boxstyle='round', pad=0.5))
ax1.set_xscale('log')
ax1.set_xlabel(r'$\theta$ [arcmin]')
ax1.set_ylabel(r'$\alpha(\theta)$')
# plt.savefig('plots/alpha_sys_%s.pdf' %ver,bbox_inches='tight')
plt.show()



### Plot the $\tau(\theta)$ statistics

In [ ]:
tau_stats_fname = f"{cosmoval_data_dir}/rho_tau_stats/tau_stats_{ver}.fits"
print(f"Reading tau stats from {tau_stats_fname}")
tau_stats_hdu = fits.open(tau_stats_fname)

theta = tau_stats_hdu[1].data['theta']

tau_0_p = tau_stats_hdu[1].data['tau_0_p']
vartau_0_p = tau_stats_hdu[1].data['vartau_0_p']

tau_2_p = tau_stats_hdu[1].data['tau_2_p']
vartau_2_p = tau_stats_hdu[1].data['vartau_2_p']

In [ ]:
#Plot the xi_plus
ax1 = plt.subplot(111)
ax1.tick_params(axis='both', which='both', direction='in', length=6, width=1,
                top=True, bottom=True, left=True, right=True)
ax1.yaxis.minorticks_on()

# ax1.errorbar(theta, tau_0_p*1e4, yerr=np.sqrt(vartau_0_p)*1e4, 
#                 fmt='o', markersize=6, capsize=2, capthick=1.5, ls = 'solid', lw=1.8,
#                 color='royalblue')
ax1.plot(theta, tau_0_p*1e4, marker='o', markersize=4, ls = 'solid', lw=1.8, color="royalblue")
ax1.fill_between(theta, (tau_0_p-np.sqrt(vartau_0_p))*1e4, (tau_0_p+np.sqrt(vartau_0_p))*1e4, color="powderblue", alpha=0.7)
ax1.text(0.85   , 0.88, r'$1-1$', transform=ax1.transAxes,
         bbox=dict(facecolor='white', edgecolor='black', boxstyle='round', pad=0.5))
ax1.axhline(0,ls='dashed',lw=1.5, color='grey')
# ax1.axvspan(0,10,color='gray', alpha=0.3)
# ax1.axvspan(150,200,color='gray', alpha=0.3)
ax1.set_xscale('log')
ax1.set_xlabel(r'$\theta$ [arcmin]')
ax1.set_ylabel(r'$\tau_{0,+}\times 10^4$')
# plt.savefig('plots/tau0_%s.pdf' %ver,bbox_inches='tight')
plt.show()

# Plot the xi_minus
ax2 = plt.subplot(111)
ax2.tick_params(axis='both', which='both', direction='in', length=6, width=1,
                top=True, bottom=True, left=True, right=True)
ax2.yaxis.minorticks_on()

# ax2.errorbar(theta, tau_2_p*1e4, yerr=np.sqrt(vartau_2_p)*1e4, 
#                 fmt='o', markersize=6, capsize=2, capthick=1.5, ls = 'solid', lw=1.8,
#                 color='orangered')
ax2.plot(theta, tau_2_p*1e4, marker='o', markersize=4, ls = 'solid', lw=1.8, color="orangered")
ax2.fill_between(theta, (tau_2_p-np.sqrt(vartau_2_p))*1e4, (tau_2_p+np.sqrt(vartau_2_p))*1e4, color="pink", alpha=0.7)
ax2.text(0.85, 0.88, r'$1-1$', transform=ax2.transAxes,
         bbox=dict(facecolor='white', edgecolor='black', boxstyle='round', pad=0.5))
ax2.axhline(0,ls='dashed',lw=1.5, color='grey')
# ax2.axvspan(0,10,color='gray', alpha=0.3)
# ax2.axvspan(150,200,color='gray', alpha=0.3)
ax2.set_xscale('log')
ax2.set_xlabel(r'$\theta$ [arcmin]')
ax2.set_ylabel(r'$\tau_{2,+}\times 10^4$')
# plt.savefig('plots/tau2_%s.pdf' %ver,bbox_inches='tight')
plt.show()


### Plot the $\tau$ covariance matrix

In [ ]:
tau_covmat_file = f"{cosmoval_data_dir}/rho_tau_stats/cov_tau_{ver}_th.npy"
print(f"Reading tau covmat from {tau_covmat_file}")
tau_cov = np.load(tau_covmat_file)
ndata = len(tau_cov[0])
tau_cov = tau_cov[:int(2*ndata/3),:int(2*ndata/3)]
cmap = 'coolwarm'

In [ ]:
ndata = len(tau_cov[0])
pp_norm = np.zeros((ndata,ndata))
for i in range(ndata):
    for j in range(ndata):
        pp_norm[i][j] = tau_cov[i][j]/ np.sqrt(tau_cov[i][i]*tau_cov[j][j])

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1) 
extent = (0, ndata, ndata, 0)
im3 = ax.imshow(pp_norm, cmap=cmap, vmin=-1, vmax=1, extent=extent)

plt.axvline(x=int(ndata/2),color='black',linewidth=1.0)
plt.axhline(y=int(ndata/2),color='black',linewidth=1.0)

fig.colorbar(im3, orientation='vertical')
ticks = np.arange(0,ndata+10,10)
tick_labels = np.array([0,10,0,10,20])
ax.set_xticks(ticks,labels= tick_labels)
ax.set_yticks(ticks,labels= tick_labels)
ax.text(int(ndata/4), ndata+5, r'$\tau_{0,+}(\theta)$')
ax.text(3*int(ndata/4), ndata+5, r'$\tau_{2,+}(\theta)$')
ax.text(-11, int(ndata/4), r'$\tau_{0,+}(\theta)$')
ax.text(-11, 3*int(ndata/4), r'$\tau_{2,+}(\theta)$')

# plt.savefig("plots/tau_covmat.pdf", bbox_inches='tight')
plt.show()